In [1]:
#tweet cleaning
#https://www.kaggle.com/ragnisah/text-data-cleaning-tweets-analysis
#https://towardsdatascience.com/basic-data-cleaning-engineering-session-twitter-sentiment-data-95e5bd2869ec

#Topic Modeilling
#https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

#Sentiment Analysis
#https://towardsdatascience.com/social-media-sentiment-analysis-49b395771197

#Cleaning
#Puncuation
#lower case
#stopwords
#abbreviations

# Importing modules
import pandas as pd
import os

# Load the regular expression library
import re

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk as nltk
nltk.download('stopwords')
nltk.download('wordnet')
import gensim
from gensim.utils import simple_preprocess
from gensim.models.wrappers import LdaMallet
from nltk.corpus import stopwords
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import pyLDAvis.gensim
import pickle 
import pyLDAvis
#import spacy
#from scispacy.abbreviation import AbbreviationDetector

#sentiment analysis
#https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d
import sqlite3
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\megha\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [2]:
# Import the wordcloud library
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator

C:\Users\megha\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\megha\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
diabetes2018 = pd.read_csv('all_diabetes2018.csv')

In [5]:
# Print head
diabetes2018.head(10)

,Post ID,Author,Text,Score,Publish Date,Permalink,Flair,Total No. of Comments,Title,Url
0,7ng7rl,penguinsarefun,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,5,2018-01-01 9:16,/r/diabetes/comments/7ng7rl/t2_newbie/,Type 2,2.0,T2 Newbie,https://www.reddit.com/r/diabetes/comments/7ng7rl/t2_newbie/
1,7nlbda,MrCanoe,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",2,2018-01-02 0:58,/r/diabetes/comments/7nlbda/looking_into_keto_diet_how_serious_are_risks_of/,Type 2,6.0,"Looking into Keto diet, how serious are risks of hypoglycemia?",https://www.reddit.com/r/diabetes/comments/7nlbda/looking_into_keto_diet_how_serious_are_risks_of/
2,7nnmor,Heylola2,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,12,2018-01-02 8:55,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/,Type 2,7.0,Great start to the new year!!,https://www.reddit.com/r/diabetes/comments/7nnmor/great_start_to_the_new_year/
3,dwcllds,[deleted],"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",1,2018-03-26 22:23,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dwcllds/,NaN,NaN,NaN,NaN
4,dzokqh6,Heylola2,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,1,2018-05-27 16:56,/r/diabetes/comments/7nnmor/great_start_to_the_new_year/dzokqh6/,NaN,NaN,NaN,NaN
5,7nnogn,jdong4321,"My mom has diabetes and I check a lot of the symptoms of type 2, and have for a few months now b...",1,2018-01-02 9:02,/r/diabetes/comments/7nnogn/i_think_i_have_diabetes_but_im_overseas/,Type 2,6.0,"I think I have diabetes but I'm overseas travelling for another 45 days, how urgent is it?",https://www.reddit.com/r/diabetes/comments/7nnogn/i_think_i_have_diabetes_but_im_overseas/
6,7npao8,douglas4321,"I am type 2 not on insulin, my doctor has told me to take it before bed. I have heard people say...",1,2018-01-02 12:51,/r/diabetes/comments/7npao8/when_is_the_best_time_to_take_my_10mg_jardiance/,Type 2,2.0,when is the best time to take my 10mg jardiance Pill?,https://www.reddit.com/r/diabetes/comments/7npao8/when_is_the_best_time_to_take_my_10mg_jardiance/
7,7nr0q2,BearFan34,NaN,49,2018-01-02 17:04,/r/diabetes/comments/7nr0q2/researchers_just_accidentally_found_an_effective/,Type 2,5.0,Researchers just accidentally found an effective Alzheimer’s treatment while trying to cure diab...,http://bgr.com/2018/01/01/alzheimers-treatment-drug-type-2-diabetes-research/
8,7ntfa9,SeanP93,"Hey y’all,\n\nI’m 24-years-old, a couple pounds shy of 140 pounds, and if a stranger saw me they...",7,2018-01-03 0:43,/r/diabetes/comments/7ntfa9/new_young_type_ii_here_looking_for_help_guidance/,Type 2,17.0,"New, young type II here.. looking for help, guidance, advice, and comfort.",https://www.reddit.com/r/diabetes/comments/7ntfa9/new_young_type_ii_here_looking_for_help_guidance/
9,7nuxr5,Mike3400,NaN,18,2018-01-03 6:23,/r/diabetes/comments/7nuxr5/seen_this/,Type 2,2.0,Seen this?,http://reddit.com/r/Health/comments/7nua3p


In [6]:
list(diabetes2018.columns)

['Post ID',
 'Author',
 'Text',
 'Score',
 'Publish Date',
 'Permalink',
 'Flair',
 'Total No. of Comments',
 'Title',
 'Url']

In [7]:
#drop unnecessary columns
diabetes2018 = diabetes2018.drop(columns = ['Author', 'Score', 'Permalink', 'Flair', 'Total No. of Comments', 'Title', 'Url'])
diabetes2018

,Post ID,Text,Publish Date
0,7ng7rl,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,2018-01-01 9:16
1,7nlbda,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",2018-01-02 0:58
2,7nnmor,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,2018-01-02 8:55
3,dwcllds,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",2018-03-26 22:23
4,dzokqh6,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,2018-05-27 16:56
...,...,...,...
12514,ecv4dtv,"In 6 mos, my A1c went from 10.5-ish to 6.7, my cholesterol down 30 pts, triglycerides down 90 pt...",2018-12-30 2:02
12515,ecve2zu,"What everyone else said. I don't even do full keto - I keep my carbs under 50, instead of under ...",2018-12-30 5:56
12516,ecwd91h,A perfect example of not letting the perfect become the enemy of the good.,2018-12-30 13:32
12517,ecwiejq,Same. Mine was 9.3. Was down to 5.5 after 90 days and been 5.2 and 5.3 ever since(about a year s...,2018-12-30 14:31


In [8]:
#remove any blank texts
diabetes2018 = diabetes2018[diabetes2018['Text'].notna()]
diabetes2018.head(10)

,Post ID,Text,Publish Date
0,7ng7rl,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,2018-01-01 9:16
1,7nlbda,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",2018-01-02 0:58
2,7nnmor,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,2018-01-02 8:55
3,dwcllds,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",2018-03-26 22:23
4,dzokqh6,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,2018-05-27 16:56
5,7nnogn,"My mom has diabetes and I check a lot of the symptoms of type 2, and have for a few months now b...",2018-01-02 9:02
6,7npao8,"I am type 2 not on insulin, my doctor has told me to take it before bed. I have heard people say...",2018-01-02 12:51
8,7ntfa9,"Hey y’all,\n\nI’m 24-years-old, a couple pounds shy of 140 pounds, and if a stranger saw me they...",2018-01-03 0:43
10,7ny6jm,I made a post [here](https://www.reddit.com/r/diabetes/comments/7jfqbt/types_2_when_metformin_an...,2018-01-03 14:11
11,7o0j4z,I'm a recently diagnosed Type 2 Diabetic (Nov 2017). My prescription coverage dropped Jardiance ...,2018-01-03 20:26


In [9]:
#remove any blank texts
diabetes2018 = diabetes2018[diabetes2018['Text'].notna()]
diabetes2018.head(10)

,Post ID,Text,Publish Date
0,7ng7rl,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,2018-01-01 9:16
1,7nlbda,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",2018-01-02 0:58
2,7nnmor,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,2018-01-02 8:55
3,dwcllds,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",2018-03-26 22:23
4,dzokqh6,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,2018-05-27 16:56
5,7nnogn,"My mom has diabetes and I check a lot of the symptoms of type 2, and have for a few months now b...",2018-01-02 9:02
6,7npao8,"I am type 2 not on insulin, my doctor has told me to take it before bed. I have heard people say...",2018-01-02 12:51
8,7ntfa9,"Hey y’all,\n\nI’m 24-years-old, a couple pounds shy of 140 pounds, and if a stranger saw me they...",2018-01-03 0:43
10,7ny6jm,I made a post [here](https://www.reddit.com/r/diabetes/comments/7jfqbt/types_2_when_metformin_an...,2018-01-03 14:11
11,7o0j4z,I'm a recently diagnosed Type 2 Diabetic (Nov 2017). My prescription coverage dropped Jardiance ...,2018-01-03 20:26


In [12]:
#remove any duplicate posts
diabetes2018 = diabetes2018.drop_duplicates(subset=['Text'])

In [13]:
#move date column
diabetes2018 = diabetes2018[['Post ID', 'Publish Date', 'Text']]
diabetes2018

,Post ID,Publish Date,Text
0,7ng7rl,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...
1,7nlbda,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a..."
2,7nnmor,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...
3,dwcllds,2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de..."
4,dzokqh6,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...
...,...,...,...
12514,ecv4dtv,2018-12-30 2:02,"In 6 mos, my A1c went from 10.5-ish to 6.7, my cholesterol down 30 pts, triglycerides down 90 pt..."
12515,ecve2zu,2018-12-30 5:56,"What everyone else said. I don't even do full keto - I keep my carbs under 50, instead of under ..."
12516,ecwd91h,2018-12-30 13:32,A perfect example of not letting the perfect become the enemy of the good.
12517,ecwiejq,2018-12-30 14:31,Same. Mine was 9.3. Was down to 5.5 after 90 days and been 5.2 and 5.3 ever since(about a year s...


In [14]:
#sentiment analysis with textblob
#https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d

#load the descriptions into textblob
desc_blob = [TextBlob(desc) for desc in diabetes2018['Text']]

#add the sentiment metrics to the dataframe
diabetes2018['tb_Pol'] = [b.sentiment.polarity for b in desc_blob]
diabetes2018['tb_Subj'] = [b.sentiment.subjectivity for b in desc_blob]

diabetes2018.head()

,Post ID,Publish Date,Text,tb_Pol,tb_Subj
0,7ng7rl,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,0.112973,0.552652
1,7nlbda,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",0.005572,0.433556
2,7nnmor,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,0.201414,0.588510
3,dwcllds,2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",0.095000,0.437778
4,dzokqh6,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,-0.081250,0.702500


In [15]:
#load VADER
analyzer = SentimentIntensityAnalyzer()

#Add VADER metrics to dataframe
diabetes2018['compound'] = [analyzer.polarity_scores(v)['compound'] for v in diabetes2018['Text']]
diabetes2018['neg'] = [analyzer.polarity_scores(v)['neg'] for v in diabetes2018['Text']]
diabetes2018['neu'] = [analyzer.polarity_scores(v)['neu'] for v in diabetes2018['Text']]
diabetes2018['pos'] = [analyzer.polarity_scores(v)['pos'] for v in diabetes2018['Text']]

diabetes2018.head()

,Post ID,Publish Date,Text,tb_Pol,tb_Subj,compound,neg,neu,pos
0,7ng7rl,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,0.112973,0.552652,0.5319,0.087,0.797,0.116
1,7nlbda,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",0.005572,0.433556,-0.9600,0.108,0.846,0.046
2,7nnmor,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,0.201414,0.588510,0.9133,0.071,0.787,0.142
3,dwcllds,2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",0.095000,0.437778,0.8477,0.048,0.764,0.189
4,dzokqh6,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,-0.081250,0.702500,0.0000,0.000,1.000,0.000


In [16]:
diabetes2018[["Publish Date", "Text", "compound", "neg", "neu", "pos"]].head(50)

,Publish Date,Text,compound,neg,neu,pos
0,2018-01-01 9:16,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,0.5319,0.087,0.797,0.116
1,2018-01-02 0:58,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",-0.9600,0.108,0.846,0.046
2,2018-01-02 8:55,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,0.9133,0.071,0.787,0.142
3,2018-03-26 22:23,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",0.8477,0.048,0.764,0.189
4,2018-05-27 16:56,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,0.0000,0.000,1.000,0.000
5,2018-01-02 9:02,"My mom has diabetes and I check a lot of the symptoms of type 2, and have for a few months now b...",-0.0772,0.070,0.861,0.069
6,2018-01-02 12:51,"I am type 2 not on insulin, my doctor has told me to take it before bed. I have heard people say...",0.6908,0.000,0.900,0.100
8,2018-01-03 0:43,"Hey y’all,\n\nI’m 24-years-old, a couple pounds shy of 140 pounds, and if a stranger saw me they...",0.8966,0.087,0.816,0.097
10,2018-01-03 14:11,I made a post [here](https://www.reddit.com/r/diabetes/comments/7jfqbt/types_2_when_metformin_an...,-0.5423,0.056,0.899,0.045
11,2018-01-03 20:26,I'm a recently diagnosed Type 2 Diabetic (Nov 2017). My prescription coverage dropped Jardiance ...,-0.0679,0.061,0.868,0.071


In [17]:
diabetes2018[['Text','tb_Pol', 'tb_Subj']].head(50)

,Text,tb_Pol,tb_Subj
0,Hey all! Happy New year's! Just wanted to say hi. I was diagnosed with type 2 DM on Thursday. ...,0.112973,0.552652
1,"Hello I'm a 6'1, 260lbs male with Type 2 diabetes. I've been looking into Keto for a while now a...",0.005572,0.433556
2,I had my a1c checked today and it's at 5.1!!! Three/four months ago it was at 7.9! \nI'm being t...,0.201414,0.588510
3,"Hey this is an old post but congrats! This is very motivating.\n\nSide note, how much do you de...",0.095000,0.437778
4,I stick my fitbit on my ankle to count 'steps' so I can end up using it the entire time I'm at m...,-0.081250,0.702500
5,"My mom has diabetes and I check a lot of the symptoms of type 2, and have for a few months now b...",-0.111612,0.387875
6,"I am type 2 not on insulin, my doctor has told me to take it before bed. I have heard people say...",0.550000,0.350000
8,"Hey y’all,\n\nI’m 24-years-old, a couple pounds shy of 140 pounds, and if a stranger saw me they...",0.116237,0.524385
10,I made a post [here](https://www.reddit.com/r/diabetes/comments/7jfqbt/types_2_when_metformin_an...,-0.040314,0.371266
11,I'm a recently diagnosed Type 2 Diabetic (Nov 2017). My prescription coverage dropped Jardiance ...,0.002500,0.590000
